### Installing and Importing Required Modules

In [ ]:
# ✅ One-time setup
!pip install opencv-python pillow

import cv2
import numpy as np
import io
import base64
from PIL import Image
from IPython.display import Javascript, display
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow

### ✅ JavaScript to capture image from webcam

In [ ]:
def capture_image_from_camera():
    js = Javascript('''
    async function takePhoto() {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = '📷 Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getTracks().forEach(t => t.stop());
      div.remove();
      return canvas.toDataURL('image/jpeg');
    }
    takePhoto();
    ''')
    display(js)
    data = eval_js('takePhoto()')
    binary = io.BytesIO(base64.b64decode(data.split(',')[1]))
    img = Image.open(binary)
    return cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)

### Cartoonify Function

In [ ]:
def cartoonify_image(img):
    """Cartoonifier: clean edges + smooth color + high quality."""
    # Image Resize
    img = cv2.resize(img, (960, 720))

    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray_blur = cv2.medianBlur(gray, 5)

    # Adaptive Threshold for edge mask
    edges = cv2.adaptiveThreshold(gray_blur, 255,
                                  cv2.ADAPTIVE_THRESH_MEAN_C,
                                  cv2.THRESH_BINARY,
                                  blockSize=7,
                                  C=5)

    # Smooth the color image
    color = cv2.bilateralFilter(img, d=9, sigmaColor=180, sigmaSpace=300)

    # Combine edges with color image
    cartoon = cv2.bitwise_and(color, color, mask=edges)

    # Optional: soft comic blur
    cartoon = cv2.GaussianBlur(cartoon, (3, 3), 0)

    return cartoon

### ✅ Run everything

In [ ]:
img = capture_image_from_camera()
cartoon = cartoonify_image(img)

### ✅ Show output

In [ ]:
cv2_imshow(cartoon)


### ✅ Save high-quality output

In [ ]:
cv2.imwrite("cartoonified_camera_output.jpg", cartoon, [cv2.IMWRITE_JPEG_QUALITY, 100])
print("✅ Saved as cartoonified_camera_output.jpg")